In [1]:
import random
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split , RandomizedSearchCV
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import csv
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# Feature options
# Define categories
moods = ['Happy', 'Lazy', 'Excited', 'Sad', 'Angry', 'Neutral', 'Romantic']
genders = ['Male', 'Female']
weathers = ['Cold', 'Mild', 'Warm']
styles = ['Casual', 'Formal', 'Sporty', 'Ethnic', 'Trendy']


In [3]:
outfit_map = {
    ('Happy', 'Male'): [
        'Light cotton shirt with shorts', 'Blazer & Jeans', 'Sporty Tracksuit',
        'Formal shirt and trousers', 'Warm hoodie and jeans'
    ],
    ('Happy', 'Female'): [
        'Summer dress with sandals', 'Trendy crop top and jeans', 'Kurti with palazzos',
        'Ethnic saree with blouse', 'Warm hoodie and jeans'
    ],
    ('Lazy', 'Male'): [
        'Warm hoodie and jeans', 'Casual t-shirt and shorts', 'Sweatpants and sweatshirt'
    ],
    ('Lazy', 'Female'): [
        'Cozy sweater with leggings', 'Loose kurti with palazzos', 'Sweatpants and hoodie'
    ],
    ('Excited', 'Male'): [
        'Blazer & Jeans', 'Sporty Tracksuit', 'Formal shirt and trousers'
    ],
    ('Excited', 'Female'): [
        'Trendy crop top and jeans', 'Summer dress with sandals', 'Ethnic saree with blouse'
    ],
    ('Sad', 'Male'): [
        'Warm sweater with jeans', 'Dark leather jacket and ripped jeans'
    ],
    ('Sad', 'Female'): [
        'Warm sweater with jeans', 'Loose kurti with palazzos'
    ],
    ('Angry', 'Male'): [
        'Dark leather jacket and ripped jeans', 'Black t-shirt with cargo pants'
    ],
    ('Angry', 'Female'): [
        'Black leather jacket and ripped jeans', 'Dark casual dress'
    ],
    ('Neutral', 'Male'): [
        'Casual t-shirt and chinos', 'Sweatshirt and jeans'
    ],
    ('Neutral', 'Female'): [
        'Simple kurti with palazzos', 'Casual t-shirt and jeans'
    ],
    ('Romantic', 'Male'): [
        'Formal shirt and trousers', 'Casual blazer and jeans'
    ],
    ('Romantic', 'Female'): [
        'Elegant floral dress', 'Ethnic saree with blouse'
    ]
}

In [4]:
# Generate 500 rows
rows = []
for _ in range(500):
    mood = random.choice(moods)
    gender = random.choice(genders)
    weather = random.choice(weathers)
    style = random.choice(styles)

In [5]:
# Load data
df = pd.read_csv('outfit_dataset.csv')



In [6]:
# Define rare classes you want to duplicate
rare_classes = [
    'Casual t shirt and jeans',
    'Casual t shirts and shorts',
    'Sweatpants and sweatshirts',
    'Light cotton shirt with shorts',
    'Kurti with plazo',
    'Black leather jacket with ripped jeans'
]

# Duplicate rare classes to increase their frequency
duplication_factor = 3  # Change this based on how many times you want to duplicate
for rc in rare_classes:
    rare_samples = df[df['Outfit Label'] == rc]
    if not rare_samples.empty:
        df = pd.concat([df] + [rare_samples]*duplication_factor, ignore_index=True)


In [7]:
# Initialize encoders
le_mood = LabelEncoder()
le_gender = LabelEncoder()
le_weather = LabelEncoder()
le_style = LabelEncoder()
le_outfit = LabelEncoder()

# Encode features with separate encoders
df['Mood_enc'] = le_mood.fit_transform(df['Mood'])
df['Gender_enc'] = le_gender.fit_transform(df['Gender'])
df['Weather_enc'] = le_weather.fit_transform(df['Weather'])
df['Style_enc'] = le_style.fit_transform(df['Style'])


In [8]:
# Encode target label (Outfit Label)
df['Outfit_enc'] = le_outfit.fit_transform(df['Outfit Label'])

# Prepare feature matrix X and target y
X = df[['Mood_enc', 'Gender_enc', 'Weather_enc', 'Style_enc']]
y = df['Outfit_enc']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [9]:
# Prepare a single example input (example: Happy, Female, Warm, Casual)
sample = pd.DataFrame({
    'Mood_enc': le_mood.transform(['Happy']),
    'Gender_enc': le_gender.transform(['Female']),
    'Weather_enc': le_weather.transform(['Warm']),
    'Style_enc': le_style.transform(['Casual'])
})

In [10]:
# Predict outfit
pred_enc = model.predict(sample)[0]
pred_outfit = le_outfit.inverse_transform([pred_enc])[0]

print("Predicted outfit:", pred_outfit)

Predicted outfit: Trendy crop top and jeans


In [11]:
# Predict on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Model Accuracy: 50.48%


In [12]:
import joblib

# After fitting encoders and training model:

joblib.dump(le_mood, 'le_mood.pkl')
joblib.dump(le_gender, 'le_gender.pkl')
joblib.dump(le_weather, 'le_weather.pkl')
joblib.dump(le_style, 'le_style.pkl')
joblib.dump(le_outfit, 'le_outfit.pkl')

joblib.dump(model, 'outfit_model.pkl')


['outfit_model.pkl']